In [1]:
import pandas as pd
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
data = pd.read_csv('data/occurrence_data.csv', delimiter='\t')
original_data = data.copy()
columns = data.columns

/tmp/ipykernel_746317/1039270105.py:1: DtypeWarning: Columns (45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/occurrence_data.csv', delimiter='\t')


In [3]:
data = data.loc[:,['gbifID', 'countryCode', 'locality',
       'stateProvince', 'decimalLatitude', 'decimalLongitude', 'elevation',
       'eventDate', 'day',
       'month', 'year']]
data = data.drop(15365)

In [4]:
nan_values = data[['day', 'month', 'year']].isna()
drop_nans = data[nan_values.any(axis=1)].index
data = data.drop(drop_nans)
date_columns = data.columns[-3:]
data[date_columns] = data[date_columns].astype(int)

In [5]:
data_loc_date = data[['decimalLatitude', 'decimalLongitude', 'day',
       'month', 'year']]
data_loc_date.columns = ['lat', 'lon', 'day',
       'month', 'year']
nan_vals = data_loc_date.isna()
drop_index = nan_vals[nan_vals.any(axis=1)].index
data_loc_date = data_loc_date.drop(drop_index)
data_loc_date = data_loc_date[data_loc_date['year']>=1960]


In [14]:
csv_out = data_loc_date.copy()
csv_out['id'] = csv_out.index +1
csv_out['date'] = csv_out['year'].astype(str) + '_' + csv_out['month'].astype(str) + '_' + csv_out['day'].astype(str)

In [23]:
csv_out = csv_out.drop(16645)

In [24]:
csv_out = csv_out[['lat', 'lon']]
csv_out.to_csv('points.csv')

In [6]:
base_dir = 'data/tifs'

tif_data = []

for dirpath, dirnames, filenames in os.walk(base_dir):
    for file in filenames:
        if file.endswith('.tif'):
            # Extract year and month from the filename
            year, month = file.split('_')[-1].split('.')[0].split('-')
            data_type = file.split('_')[-2]
            
            # Append to data list
            tif_data.append({
                'file_path': os.path.join(dirpath, file),
                'type': data_type,
                'year': int(year),
                'month': int(month)
            })

df_tifs = pd.DataFrame(tif_data)

df_tifs

,file_path,type,year,month
0,data/tifs/wc2.1_cruts4.06_2.5m_prec_1990-1999/...,prec,1998,2
1,data/tifs/wc2.1_cruts4.06_2.5m_prec_1990-1999/...,prec,1998,9
2,data/tifs/wc2.1_cruts4.06_2.5m_prec_1990-1999/...,prec,1993,9
3,data/tifs/wc2.1_cruts4.06_2.5m_prec_1990-1999/...,prec,1999,5
4,data/tifs/wc2.1_cruts4.06_2.5m_prec_1990-1999/...,prec,1996,4
...,...,...,...,...
2227,data/tifs/wc2.1_cruts4.06_2.5m_tmin_2000-2009/...,tmin,2001,5
2228,data/tifs/wc2.1_cruts4.06_2.5m_tmin_2000-2009/...,tmin,2003,1
2229,data/tifs/wc2.1_cruts4.06_2.5m_tmin_2000-2009/...,tmin,2005,12
2230,data/tifs/wc2.1_cruts4.06_2.5m_tmin_2000-2009/...,tmin,2001,2


In [7]:
# import concurrent.futures

# def get_value_from_tif(lat, long, file_path):
#     with rasterio.open(file_path) as src:
#         row, col = src.index(long, lat)
#         value = src.read(1)[row, col]
#     return value

# def process_row(row, df_tifs):
#     results = {}
    
#     for data_type in ['prec', 'tmin', 'tmax']:
#         file_path = df_tifs[
#             (df_tifs['type'] == data_type) &
#             (df_tifs['year'] == row['year']) &
#             (df_tifs['month'] == row['month'])
#         ]['file_path'].iloc[0]
        
#         try:
#             results[data_type] = get_value_from_tif(row['lat'], row['lon'], file_path)
#         except:
#             print('could not get value for ', row['lat'], row['lon'], file_path)
#             results[data_type] = None
    
#     return results

# data_loc_date['prec'] = None
# data_loc_date['tmin'] = None
# data_loc_date['tmax'] = None

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     futures = {executor.submit(process_row, row, df_tifs): row for _, row in data_loc_date.iterrows()}
    
#     for future in concurrent.futures.as_completed(futures):
#         row = futures[future]
#         data = future.result()
#         idx = row.name
        
#         data_loc_date.at[idx, 'prec'] = data['prec']
#         data_loc_date.at[idx, 'tmin'] = data['tmin']
#         data_loc_date.at[idx, 'tmax'] = data['tmax']



In [6]:
import os
import rasterio
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def get_value_from_tif(lat, long, file_path):
    with rasterio.open(file_path) as src:
        row, col = src.index(long, lat)
        value = src.read(1)[row, col]
    return value

directory = '/home/mpgetz/repos/bee_mapping/data/wc2.1_30s_bio'
all_files = [f for f in os.listdir(directory) if f.endswith('.tif') and "wc2.1_30s_" in f]

for file in all_files:
    column_name = file.split("wc2.1_30s_")[1].replace(".tif", "")

    data_loc_date[column_name] = None
    
    file_path = os.path.join(directory, file)
    
    for index, row in data_loc_date.iterrows():
        try:
            value = get_value_from_tif(row['lat'], row['lon'], file_path)
            data_loc_date.at[index, column_name] = value
        except:
            print(f'could not get value for {row["lat"]}, {row["lon"]} from file {file}')
    break

data_loc_date.to_csv('test.csv')

could not get value for 50.388493, 8.898926 from file wc2.1_30s_bio_11.tif


In [ ]:
def get_value_from_tif(lat, long, file_path):
    try:
        with rasterio.open(file_path) as src:
            row, col = src.index(long, lat)
            return src.read(1)[row, col]
    except Exception as e:
        print(f'Error at {lat}, {long}: {e}')
        return None

directory = '/home/mpgetz/repos/bee_mapping/data/wc2.1_30s_bio'
all_files = [f for f in os.listdir(directory) if f.endswith('.tif') and "wc2.1_30s_" in f]

def process_file(file):
    column_name = file.split("wc2.1_30s_")[1].replace(".tif", "")
    file_path = os.path.join(directory, file)
    data_loc_date[column_name] = data_loc_date.apply(lambda row: get_value_from_tif(row['lat'], row['lon'], file_path), axis=1)

with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
    executor.map(process_file, all_files)

data_loc_date.to_csv('test.csv')

In [6]:
import os
import rasterio
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def get_value_from_array(lat, long, src, array):
    try:
        row, col = src.index(long, lat)
        return array[row, col]
    except Exception as e:
        print(f'Error at {lat}, {long}: {e}')
        return None

directory = '/home/mpgetz/repos/bee_mapping/data/wc2.1_30s_bio'
all_files = [f for f in os.listdir(directory) if f.endswith('.tif') and "wc2.1_30s_" in f]

print(all_files)

def process_file(file):
    column_name = file.split("wc2.1_30s_")[1].replace(".tif", "")
    file_path = os.path.join(directory, file)
    print(f'Processing: {column_name}\n')

    with rasterio.open(file_path) as src:
        array = src.read(1)  # Read the entire file into an array
        data_loc_date[column_name] = data_loc_date.apply(lambda row: get_value_from_array(row['lat'], row['lon'], src, array), axis=1)

# Assuming data_loc_date is your DataFrame with lat and lon
with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust max_workers based on your CPU
    executor.map(process_file, all_files)

data_loc_date.to_csv('test2.csv')


['wc2.1_30s_bio_11.tif', 'wc2.1_30s_bio_3.tif', 'wc2.1_30s_bio_9.tif', 'wc2.1_30s_bio_13.tif', 'wc2.1_30s_bio_5.tif', 'wc2.1_30s_bio_6.tif', 'wc2.1_30s_elev.tif', 'wc2.1_30s_bio_17.tif', 'wc2.1_30s_bio_10.tif', 'wc2.1_30s_bio_2.tif', 'wc2.1_30s_bio_15.tif', 'wc2.1_30s_bio_12.tif', 'wc2.1_30s_bio_1.tif', 'wc2.1_30s_bio_16.tif', 'wc2.1_30s_bio_8.tif', 'wc2.1_30s_bio_4.tif', 'wc2.1_30s_bio_19.tif', 'wc2.1_30s_bio_7.tif', 'wc2.1_30s_bio_14.tif', 'wc2.1_30s_bio_18.tif']
Processing: bio_11

Processing: bio_3

Processing: bio_9

Processing: bio_13

Processing: bio_5

Processing: bio_6

Processing: elev

Processing: bio_17



: 